In [1]:
import xarray as xr
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
from matplotlib.colors import LogNorm
from xhistogram.xarray import histogram
import os
import time
from matplotlib.colors import LogNorm
import dask

In [2]:
listOfDir = os.listdir('/Data2/SubX/')
listOfDir = listOfDir[0:-1] # drop 'CWB
listOfDir

['ESRL', 'NASA', 'EMC', 'RSMAS', 'ECCC', 'NRL', 'CESM', 'NCEP']

In [3]:
subfolder_to_try = ['FIMr1p1','GEOS_V2p1','GEFS','CCSM4','GEPS5','GEPS6','hindcast','NESM','46LCESM1', '30LCAM5','CFSv2']

In [ ]:
for ild in listOfDir:
    for isb in subfolder_to_try:
        if isb in os.listdir('/Data2/SubX/'+ild+'/'):
            print(os.listdir('/Data2/SubX/'+ild+'/'+isb)[0])


sic_sfc.zip
pr_sfc
JJA
tas_2m
forecast
forecast


# the list above will look for the subfolder in the folder. I have 11 possibilities, one for each model. In particular

In [ ]:
for ild in listOfDir:
    for isb in subfolder_to_try:
        if isb in os.listdir('/Data2/SubX/'+ild+'/'):
            print(ild+'/'+isb)


## each of this subfloders are organized differently (in fact the third ECCC doesn't have a name for the submodel, but directly the folder hindcast. I assume that is the GEM model)

## Unfortunately I cannot use a general loop to treat each model, but I have to write a separate script for each of them

## the name of the variables *should* be the same: cape, tas_2m, tdps_2m, wap_500, pr_sfc

# CESM/

In [ ]:
variable_names = ['pr', 'tas_2m']

In [ ]:
for ild in listOfDir[6:7]:
    for isb in subfolder_to_try:
        if isb in os.listdir('/Data2/SubX/'+ild+'/'):
            subfolder_dir = ild+'/'+isb
            print(subfolder_dir)
            print('/Data2/SubX/'+subfolder_dir)
            file_list = os.listdir('/Data2/SubX/'+subfolder_dir)
#             file_list1 = [ifl if ifl[-4:]!='.zip' else print('skip') for ifl in file_list]
            print(file_list)


In [ ]:
!ls /Data2/SubX/CESM/46LCESM1/hindcast/CESM1_46LCAM5

In [ ]:
# ds1 = xr.open_dataset('/Data2/SubX/CESM/46LCESM1/hindcast/CESM1_46LCAM5/pr/1999/01/pr_sfc_CESM1_46LCAM5_06jan1999_00z_d01_d45_m00.nc')
# ds1

In [ ]:
# ds2 = pre_process_func(ds1,'0')
# ds2

In [56]:
from glob import glob
def pre_process_func(ds,strM):
    ds.coords['TIME'] = ds.indexes['TIME'].to_datetimeindex()

    ds = ds.rename({'TIME':'time', 'LAT':'lat', 'LON':'lon'})
    ds = ds.sel(lat=slice(23,50), lon=slice(230,300))
    ds.coords['S'] = ds.time[0].values
    ds.coords['S'] = ds.S - np.timedelta64(12,'h') 
    ds.coords['M'] = np.int(strM)
    ds.coords['time'] = ds.time-ds.S
    ds = ds.expand_dims(['S','M'])
    return ds
def read_netcdfs(files, dim, transform_func=None):
#     print(files)
    def process_one_path(path):
        # use a context manager, to ensure the file gets closed after use
        with xr.open_dataset(path, chunks={}) as ds:
            # transform_func should do some sort of selection or
            # aggregation
            if transform_func is not None:
                ds = transform_func(ds, path[-5:-3])
            # load all data from the transformed dataset, to ensure we can
            # use it after closing each original file
#             print(ds)
#             ds.load()
            return ds

    paths = sorted(glob(files))
#     print(paths)
    datasets = [process_one_path(p) for p in paths]
    
    combined = xr.combine_by_coords(datasets)
    combined = combined.chunk({'S':4,'M':-1})
    return combined



var_l=[]
for ivar in variable_names:
    print(ivar)
    if ivar in os.listdir('/Data2/SubX/CESM/46LCESM1/hindcast/CESM1_46LCAM5/'):    
        c_l = []
        for iy in np.arange(1999, 2000):
            print(iy)
            # here we suppose we only care about the combined mean of each file;
            # you might also use indexing operations like .sel to subset datasets
            combined = read_netcdfs('/Data2/SubX/CESM/46LCESM1/hindcast/CESM1_46LCAM5/'+ivar+'/'+np.str(iy)+'/*/*.nc', dim=['S','M'],
                                    transform_func=pre_process_func)
            
            c_l.append(combined)
    all_values = xr.concat(c_l, dim='S') 
    var_l.append(all_values)
all_variables = xr.merge(var_l)



pr
1999


/local/ERAin/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  This is separate from the ipykernel package so we can avoid doing imports until


tas_2m
1999


/local/ERAin/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  This is separate from the ipykernel package so we can avoid doing imports until


In [57]:
all_variables

<xarray.Dataset>
Dimensions:  (M: 10, S: 52, lat: 28, lon: 71, time: 45)
Coordinates:
  * time     (time) timedelta64[ns] 0 days 12:00:00 ... 44 days 12:00:00
  * lat      (lat) float32 23.0 24.0 25.0 26.0 27.0 ... 46.0 47.0 48.0 49.0 50.0
  * lon      (lon) float32 230.0 231.0 232.0 233.0 ... 297.0 298.0 299.0 300.0
  * M        (M) int64 0 1 2 3 4 5 6 7 8 9
  * S        (S) datetime64[ns] 1999-01-06 1999-01-13 ... 1999-12-22 1999-12-29
Data variables:
    PR       (S, M, time, lat, lon) float32 dask.array<chunksize=(4, 10, 45, 28, 71), meta=np.ndarray>
    TAS      (S, M, time, lat, lon) float32 dask.array<chunksize=(4, 10, 45, 28, 71), meta=np.ndarray>

In [58]:
from glob import glob
def pre_process_func(ds,strM):
    ds.coords['TIME'] = ds.indexes['TIME'].to_datetimeindex()

    ds = ds.rename({'TIME':'time', 'LAT':'lat', 'LON':'lon'})
    ds = ds.sel(lat=slice(23,50), lon=slice(230,300))
    ds.coords['S'] = ds.time[0].values
    ds.coords['S'] = ds.S - np.timedelta64(12,'h') 
    ds.coords['M'] = np.int(strM)
    ds.coords['time'] = ds.time-ds.S
    ds = ds.expand_dims(['S','M'])
    return ds
def read_netcdfs(files, dim, transform_func=None):
#     print(files)
    def process_one_path(path):
        # use a context manager, to ensure the file gets closed after use
        with xr.open_dataset(path, chunks={}) as ds:
            # transform_func should do some sort of selection or
            # aggregation
            if transform_func is not None:
                ds = transform_func(ds, path[-5:-3])
            # load all data from the transformed dataset, to ensure we can
            # use it after closing each original file
#             print(ds)
#             ds.load()
            return ds

    paths = sorted(glob(files))
#     print(paths)
    datasets = [process_one_path(p) for p in paths]
    
    combined = xr.combine_by_coords(datasets)
    combined = combined.chunk({'S':4,'M':-1})
    return combined



var_l=[]
for ivar in variable_names:
    print(ivar)
    if ivar in os.listdir('/Data2/SubX/CESM/30LCAM5/hindcast/CESM1_30LCAM5/'):    
        c_l = []
        for iy in np.arange(1999, 2000):
            print(iy)
            # here we suppose we only care about the combined mean of each file;
            # you might also use indexing operations like .sel to subset datasets
            combined = read_netcdfs('/Data2/SubX/CESM/30LCAM5/hindcast/CESM1_30LCAM5/'+ivar+'/'+np.str(iy)+'/*/*.nc', dim=['S','M'],
                                    transform_func=pre_process_func)
            
            c_l.append(combined)
    all_values = xr.concat(c_l, dim='S') 
    var_l.append(all_values)
all_variables = xr.merge(var_l)



pr
1999


/local/ERAin/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  This is separate from the ipykernel package so we can avoid doing imports until


tas_2m
1999


/local/ERAin/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  This is separate from the ipykernel package so we can avoid doing imports until


In [59]:
all_variables

<xarray.Dataset>
Dimensions:  (M: 10, S: 52, lat: 28, lon: 71, time: 45)
Coordinates:
  * time     (time) timedelta64[ns] 0 days 12:00:00 ... 44 days 12:00:00
  * lat      (lat) float32 23.0 24.0 25.0 26.0 27.0 ... 46.0 47.0 48.0 49.0 50.0
  * lon      (lon) float32 230.0 231.0 232.0 233.0 ... 297.0 298.0 299.0 300.0
  * M        (M) int64 0 1 2 3 4 5 6 7 8 9
  * S        (S) datetime64[ns] 1999-01-06 1999-01-13 ... 1999-12-22 1999-12-29
Data variables:
    PR       (S, M, time, lat, lon) float32 dask.array<chunksize=(4, 10, 45, 28, 71), meta=np.ndarray>
    TAS      (S, M, time, lat, lon) float32 dask.array<chunksize=(4, 10, 45, 28, 71), meta=np.ndarray>